Importing open ai gym and the package

In [4]:
import gym
import numpy as np
import airline
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env

Define the function for evaluating the model on a given environment for a specified number of episodes. 

In [19]:
def evaluate(model, env, numiters):
    rewards = []
    for j in range(numiters):
        obs = env.reset()
        tot_reward = 0
        for i in range(env.tau):
            action, states = model.predict(obs)
            _, reward, _, _ = env.step(action)
            tot_reward += reward
        rewards.append(tot_reward)
    return np.mean(rewards), np.std(rewards)

# Model is the random policy
def policy(env):
    return env.action_space.sample()
    

Setting up the configuration for the environment

In [6]:
A = np.asarray([[1, 1, 0,0,0,0], [ 0,0, 1, 1, 1, 1], [ 0,0, 0,0, 1, 1] ])
tau = 23
P = np.ones((tau, A.shape[1]))/3
c = [5, 5, 5]
f = range(10, 16)
CONFIG = {'A': A, 'f': f, 'P': P, 'starting_state': c , 'tau': tau}

Making an instance of the environment

In [7]:
env = gym.make('Airline-v0', config=CONFIG)

In [8]:
check_env(env)

In [9]:
model = A2C("MlpPolicy", env, n_steps= tau, verbose=1)
model.learn(total_timesteps =250000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-------------------------------------
| time/                 |           |
|    fps                | 597       |
|    iterations         | 100       |
|    time_elapsed       | 3         |
|    total_timesteps    | 2300      |
| train/                |           |
|    entropy_loss       | -4.13     |
|    explained_variance | 0         |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -0.000766 |
|    value_loss         | 4.38e-08  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 591       |
|    iterations         | 200       |
|    time_elapsed       | 7         |
|    total_timesteps    | 4600      |
| train/                |           |
|    entropy_loss       | -3.95     |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0007

------------------------------------
| time/                 |          |
|    fps                | 547      |
|    iterations         | 1700     |
|    time_elapsed       | 71       |
|    total_timesteps    | 39100    |
| train/                |          |
|    entropy_loss       | -3.76    |
|    explained_variance | 5.96e-08 |
|    learning_rate      | 0.0007   |
|    n_updates          | 1699     |
|    policy_loss        | -0.00888 |
|    value_loss         | 6.03e-06 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 546      |
|    iterations         | 1800     |
|    time_elapsed       | 75       |
|    total_timesteps    | 41400    |
| train/                |          |
|    entropy_loss       | -3.38    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 1799     |
|    policy_loss        | -0.0074  |
|    value_loss         | 5.66e-06 |
-

------------------------------------
| time/                 |          |
|    fps                | 514      |
|    iterations         | 3300     |
|    time_elapsed       | 147      |
|    total_timesteps    | 75900    |
| train/                |          |
|    entropy_loss       | -3.31    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 3299     |
|    policy_loss        | -0.00367 |
|    value_loss         | 1.91e-06 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 514      |
|    iterations         | 3400     |
|    time_elapsed       | 151      |
|    total_timesteps    | 78200    |
| train/                |          |
|    entropy_loss       | -3.28    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 3399     |
|    policy_loss        | -0.00373 |
|    value_loss         | 1.8e-06  |
-

------------------------------------
| time/                 |          |
|    fps                | 522      |
|    iterations         | 4900     |
|    time_elapsed       | 215      |
|    total_timesteps    | 112700   |
| train/                |          |
|    entropy_loss       | -2.64    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 4899     |
|    policy_loss        | -0.00253 |
|    value_loss         | 8.89e-07 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 523      |
|    iterations         | 5000     |
|    time_elapsed       | 219      |
|    total_timesteps    | 115000   |
| train/                |          |
|    entropy_loss       | -2.82    |
|    explained_variance | 5.96e-08 |
|    learning_rate      | 0.0007   |
|    n_updates          | 4999     |
|    policy_loss        | -0.00218 |
|    value_loss         | 8.56e-07 |
-

------------------------------------
| time/                 |          |
|    fps                | 537      |
|    iterations         | 6500     |
|    time_elapsed       | 278      |
|    total_timesteps    | 149500   |
| train/                |          |
|    entropy_loss       | -3.39    |
|    explained_variance | 5.96e-08 |
|    learning_rate      | 0.0007   |
|    n_updates          | 6499     |
|    policy_loss        | -0.00214 |
|    value_loss         | 5.19e-07 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 538      |
|    iterations         | 6600     |
|    time_elapsed       | 281      |
|    total_timesteps    | 151800   |
| train/                |          |
|    entropy_loss       | -3.42    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 6599     |
|    policy_loss        | -0.00209 |
|    value_loss         | 5.04e-07 |
-

------------------------------------
| time/                 |          |
|    fps                | 542      |
|    iterations         | 8100     |
|    time_elapsed       | 343      |
|    total_timesteps    | 186300   |
| train/                |          |
|    entropy_loss       | -3.69    |
|    explained_variance | 1.19e-07 |
|    learning_rate      | 0.0007   |
|    n_updates          | 8099     |
|    policy_loss        | -0.00186 |
|    value_loss         | 3.28e-07 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 542      |
|    iterations         | 8200     |
|    time_elapsed       | 347      |
|    total_timesteps    | 188600   |
| train/                |          |
|    entropy_loss       | -3.72    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 8199     |
|    policy_loss        | -0.00198 |
|    value_loss         | 3.2e-07  |
-

------------------------------------
| time/                 |          |
|    fps                | 544      |
|    iterations         | 9700     |
|    time_elapsed       | 409      |
|    total_timesteps    | 223100   |
| train/                |          |
|    entropy_loss       | -3       |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 9699     |
|    policy_loss        | -0.00153 |
|    value_loss         | 2.16e-07 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 544      |
|    iterations         | 9800     |
|    time_elapsed       | 413      |
|    total_timesteps    | 225400   |
| train/                |          |
|    entropy_loss       | -3.17    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 9799     |
|    policy_loss        | -0.00132 |
|    value_loss         | 2.11e-07 |
-

In [10]:
model.save("a2cTest")

In [13]:
obs = env.reset()

In [15]:
model.predict(obs)

(array([0, 0, 1, 0, 1, 0]), None)

Testing the variables

In [ ]:
print(env.state)
print(env.action_space)
print(env.observation_space) # openAI calles states observations :/

Testing the step function

In [ ]:
env.state = [1,0, 0]
print(env.step(np.asarray([1,1, 1, 1, 1, 1]), 5))
print('***')
env.state = [3, 3, 3]
print(env.step([1,1, 1, 1, 1, 1], 5))

test sample uniformly from action space at random

In [33]:
env.action_space.sample()

(array([0, 0, 1, 0, 1, 0]), None)

Simulating randomized policy

In [16]:
e = 50
rewards = []
for j in range(e):
    obs = env.reset()
    tot_reward = 0
    for i in range(tau):
        action, states_ = model.predict(obs)
        obs, reward, dones, info = env.step(action)
        tot_reward += reward
    rewards.append(tot_reward)

Test Evaluate Function

In [20]:
print('Here is the mean and standard deviation of the estimate ' + str(evaluate(model, env, 50)))

Here is the mean and standard deviation of the estimate (104.72, 9.395828861787553)
